In [4]:
# serial protocol: 
# 1. All serial protocol consists of two independent strings.
# 2. First string is one of the following:
# - p -> push solenoid
# - P -> pull solenoid
# - r -> right mov
# - l -> left mov
# - d -> down mov
# - ` -> end sub serial bundle
# * But 'p' and 'P' hasn't number string. 
#
# 3. Second string is a integer number.
#
# * So, protocol has the following forms.
# ['p', `, 'r', '23', `, 'P', `, 'r', '2', `, 'd', '1', `, 'P', `, 'l', '23', `, ...]
#
#

import cv2
import numpy as np
from collections import deque

def conv_img_to_ser_deque(img):
    threshold_val = 255
    m, n = img.shape
    ans_deque = deque()
    col = 0
    for row in img:
        row_deque = deque(row)
        cnt = 0
        flag = -1
        if col % 2 == 0:
            while row_deque:
                tmp = row_deque.popleft()
                if tmp == 0 and flag == 0:
                    cnt += 1
                elif tmp == 0:
                    if flag != -1:
                        ans_deque.append(str(cnt))
                        ans_deque.append('`')
                    ans_deque.append('P')
                    ans_deque.append('`')
                    ans_deque.append('r')
                    cnt = 1
                    flag = 0
                elif tmp == threshold_val and flag == 1:
                    cnt += 1
                elif tmp == threshold_val:
                    if flag != -1:
                        ans_deque.append(str(cnt))
                        ans_deque.append('`')
                    ans_deque.append('p')
                    ans_deque.append('`')
                    ans_deque.append('r')
                    cnt = 1
                    flag = 1
        else:
            while row_deque:
                tmp = row_deque.pop()
                if tmp == 0 and flag == 0:
                    cnt += 1
                elif tmp == 0:
                    if flag != -1:
                        ans_deque.append(str(cnt))
                        ans_deque.append('`')
                    ans_deque.append('P')
                    ans_deque.append('`')
                    ans_deque.append('l')
                    cnt = 1
                    flag = 0
                elif tmp == threshold_val and flag == 1:
                    cnt += 1
                elif tmp == threshold_val:
                    if flag != -1:
                        ans_deque.append(str(cnt))
                        ans_deque.append('`')
                    ans_deque.append('p')
                    ans_deque.append('`')
                    ans_deque.append('l')
                    cnt = 1
                    flag = 1
        if cnt == n:
            ans_deque.pop()
            ans_deque.pop()
            ans_deque.append('`')
        else:
            ans_deque.append(str(cnt))
            ans_deque.append('`')
            col += 1
        ans_deque.append('d')
        ans_deque.append('1')
        ans_deque.append('`')
    return ans_deque

def conv_ser_deque_to_img(ser_deque, img_shape):
    threshold_val = 255
    ans_img = np.zeros(img_shape)
    x, y = 0, 0
    solenoid_state = 0
    while ser_deque:
        tmp = ser_deque.popleft()
        if tmp == 'd':
            y += int(ser_deque.popleft())
        elif tmp == 'p':
            solenoid_state = 1
            ser_deque.popleft()
        elif tmp == 'P':
            solenoid_state = 0
            ser_deque.popleft()
        elif tmp == 'r':
            r_mov_dis = int(ser_deque.popleft())
            _ = ser_deque.popleft()
            while r_mov_dis != 0:
                if solenoid_state == 1:
                    ans_img[y][x] = threshold_val
                x += 1
                r_mov_dis -= 1
        elif tmp == 'l':
            l_mov_dis = int(ser_deque.popleft())
            _ = ser_deque.popleft()
            while l_mov_dis != 0:
                x -= 1
                if solenoid_state == 1:
                    ans_img[y][x] = threshold_val
                l_mov_dis -= 1
    return ans_img

if __name__ == "__main__":
    img = cv2.imread('test1.png', cv2.IMREAD_GRAYSCALE)
    img = cv2.resize(img, (0, 0), fx=1, fy=1)
    _, img = cv2.threshold(img, 80, 255, cv2.THRESH_BINARY)
    cv2.imwrite('test2.png', img)
    serial_deque = conv_img_to_ser_deque(img)

In [5]:
print(serial_deque)

deque(['P', '`', 'd', '1', '`', 'P', '`', 'd', '1', '`', 'P', '`', 'd', '1', '`', 'P', '`', 'd', '1', '`', 'P', '`', 'd', '1', '`', 'P', '`', 'd', '1', '`', 'P', '`', 'd', '1', '`', 'P', '`', 'd', '1', '`', 'P', '`', 'd', '1', '`', 'P', '`', 'd', '1', '`', 'P', '`', 'd', '1', '`', 'P', '`', 'd', '1', '`', 'P', '`', 'd', '1', '`', 'P', '`', 'd', '1', '`', 'P', '`', 'd', '1', '`', 'P', '`', 'd', '1', '`', 'P', '`', 'd', '1', '`', 'P', '`', 'd', '1', '`', 'P', '`', 'd', '1', '`', 'P', '`', 'd', '1', '`', 'P', '`', 'd', '1', '`', 'P', '`', 'd', '1', '`', 'P', '`', 'd', '1', '`', 'P', '`', 'd', '1', '`', 'P', '`', 'd', '1', '`', 'P', '`', 'd', '1', '`', 'P', '`', 'd', '1', '`', 'P', '`', 'd', '1', '`', 'P', '`', 'd', '1', '`', 'P', '`', 'd', '1', '`', 'P', '`', 'd', '1', '`', 'P', '`', 'd', '1', '`', 'P', '`', 'd', '1', '`', 'P', '`', 'd', '1', '`', 'P', '`', 'd', '1', '`', 'P', '`', 'd', '1', '`', 'P', '`', 'd', '1', '`', 'P', '`', 'd', '1', '`', 'P', '`', 'd', '1', '`', 'P', '`', 'd', '1'

In [6]:
import serial
import time
import pyautogui

def interact_ser(_str, _ard):
    _ard.write(_str.encode())
    tmp = _ard.readline()
    print(tmp.decode())
    
if __name__ == "__main__":
    port = 'COM4'  # 변동가능
    ard = serial.Serial(port, 9600)
    time.sleep(3)

    for i in serial_deque:
        if list(pyautogui.position()) != [0, 0]:
            interact_ser(i, ard)

    ard.close()


P

0

d

1

1

P

0

d

1

1

P

0

d

1

1

P

0

d

1

1

P

0

d

1

1

P

0

d

1

1

P

0

d

1

1

P

0

d

1

1

P

0

d

1

1

P

0

d

1

1

P

0

d

1

1

P

0

d

1

1

P

0

d

1

1

P

0

d

1

1

P

0

d

1

1

P

0

d

1

1

P

0

d

1

1

P

0

d

1

1

P

0

d

1

1

P

0

d

1

1

P

0

d

1

1

P

0

d

1

1

P

0

d

1

1

P

0

d

1

1

P

0

d

1

1

P

0

d

1

1

P

0

d

1

1

P

0

d

1

1

P

0

d

1

1

P

0

d

1

1

P

0

d

1

1

P

0

d

1

1

P

0

d

1

1

P

0

d

1

1

P

0

d

1

1

P

0

d

1

1

P

0

d

1

1

P

0

d

1

1

P

0

d

1

1

P

0

d

1

1

P

0

d

1

1

P

0

d

1

1

P

0

d

1

1

P

0

d

1

1

P

0

d

1

1

P

0

d

1

1

P

0

d

1

1

P

0

d

1

1

P

0

d

1

1

P

0

d

1

1

P

0

d

1

1

P

0

d

1

1

P

0

r

3

9

39

p

0

r

1

1

P

0

r

1

3

13

p

0

r

2

2

P

0

r

9

9

p

0

r

1

1

P

0

r

1

0

10

p

0

r

2

2

P

0

r

3

9

39

p

0

r

2

2

P

0

r

4

2

42

p

0

r

3

3

P

0

r

3

3

p

0


0

l

2

2

p

0

l

2

2

P

0

l

3

3

p

0

l

3

3

P

0

l

3

3

33

p

0

l

2

2

P

0

l

2

2

p

0

l

6

6

P

0

l

1

1

p

0

l

3

3

P

0

l

1

1

p

0

l

6

6

P

0

l

1

1

p

0

l

2

2

P

0

l

5

5

p

0

l

4

4

P

0

l

3

3

p

0

l

3

3

P

0

l

1

1

p

0

l

5

5

P

0

l

2

2

p

0

l

9

9

P

0

l

5

5

p

0

l

4

4

P

0

l

1

2

12

d

1

1

P

0

r

1

3

13

p

0

r

2

2

P

0

r

6

6

p

0

r

8

8

P

0

r

3

3

p

0

r

8

8

P

0

r

5

5

p

0

r

1

1

P

0

r

2

2

p

0

r

6

6

P

0

r

2

2

p

0

r

2

2

P

0

r

1

1

p

0

r

3

3

P

0

r

1

1

p

0

r

3

3

P

0

r

9

9

p

0

r

2

2

P

0

r

3

4

34

p

0

r

6

6

P

0

r

6

6

p

0

r

3

3

P

0

r

3

1

31

p

0

1

1

11

P

0

8

8

p

0

r

3

3

P

0

r

5

5

p

0

r

3

3

P

0

r

8

8

p

0

r

3

3

P

0

r

7

7

p

0

r

1

1

P

0

r

3

3

p

0

r

3

3

P

0

r

1

1

p

0

r

2

2

P

0

r

2

1

p

21

r

6

6

P

0

r

2

2

p

0

r

9

9



SerialException: WriteFile failed (PermissionError(13, '장치가 명령을 인식하지 않습니다.', None, 22))

In [ ]:
import serial

def interact_ser(_str, _ard):
    _ard.write(_str.encode())
    tmp = _ard.readline()
    print(tmp.decode())
    
if __name__ == "__main__":
    port = 'COM4'  # 변동가능
    ard = serial.Serial(port, 9600)

    while True:
        send_str = input()
        if send_str == '-':
            break
        else:
            interact_ser(send_str, ard)

    ard.close()


In [ ]:
import serial
import time
import pyautogui
from collections import deque
import cv2
import numpy as np

serial_deque = deque(['P', '`', 'd', '1', '`', 'P', '`', 'd', '1', '`', 'P', '`', 'd', '1', '`', 'P', '`', 'd', '1', '`', 'P', '`', 'd', '1', '`', 'P', '`', 'd', '1', '`', 'P', '`', 'd', '1', '`', 'P', '`', 'd', '1', '`', 'P', '`', 'd', '1', '`', 'P', '`', 'd', '1', '`', 'P', '`', 'd', '1', '`', 'P', '`', 'd', '1', '`', 'P', '`', 'd', '1', '`', 'P', '`', 'd', '1', '`', 'P', '`', 'd', '1', '`', 'P', '`', 'd', '1', '`', 'P', '`', 'd', '1', '`', 'P', '`', 'd', '1', '`', 'P', '`', 'd', '1', '`', 'P', '`', 'd', '1', '`', 'P', '`', 'd', '1', '`', 'P', '`', 'd', '1', '`', 'P', '`', 'd', '1', '`', 'P', '`', 'd', '1', '`', 'P', '`', 'd', '1', '`', 'P', '`', 'd', '1', '`', 'P', '`', 'd', '1', '`', 'P', '`', 'd', '1', '`', 'P', '`', 'd', '1', '`', 'P', '`', 'd', '1', '`', 'P', '`', 'd', '1', '`', 'P', '`', 'd', '1', '`', 'P', '`', 'd', '1', '`', 'P', '`', 'd', '1', '`', 'P', '`', 'd', '1', '`', 'P', '`', 'd', '1', '`', 'P', '`', 'd', '1', '`', 'P', '`', 'd', '1', '`', 'P', '`', 'd', '1', '`', 'P', '`', 'd', '1', '`', 'P', '`', 'd', '1', '`', 'P', '`', 'd', '1', '`', 'P', '`', 'd', '1', '`', 'P', '`', 'd', '1', '`', 'P', '`', 'd', '1', '`', 'P', '`', 'd', '1', '`', 'P', '`', 'd', '1', '`', 'P', '`', 'd', '1', '`', 'P', '`', 'd', '1', '`', 'P', '`', 'd', '1', '`', 'P', '`', 'd', '1', '`', 'P', '`', 'd', '1', '`', 'P', '`', 'r', '39', '`', 'p', '`', 'r', '1', '`', 'P', '`', 'r', '13', '`', 'p', '`', 'r', '2', '`', 'P', '`', 'r', '9', '`', 'p', '`', 'r', '1', '`', 'P', '`', 'r', '10', '`', 'p', '`', 'r', '2', '`', 'P', '`', 'r', '39', '`', 'p', '`', 'r', '2', '`', 'P', '`', 'r', '42', '`', 'p', '`', 'r', '3', '`', 'P', '`', 'r', '3', '`', 'p', '`', 'r', '2', '`', 'P', '`', 'r', '9', '`', 'p', '`', 'r', '1', '`', 'P', '`', 'r', '36', '`', 'p', '`', 'r', '1', '`', 'P', '`', 'r', '36', '`', 'p', '`', 'r', '2', '`', 'P', '`', 'r', '8', '`', 'p', '`', 'r', '2', '`', 'P', '`', 'r', '24', '`', 'p', '`', 'r', '1', '`', 'P', '`', 'r', '12', '`', 'p', '`', 'r', '2', '`', 'P', '`', 'r', '10', '`', 'p', '`', 'r', '1', '`', 'P', '`', 'r', '12', '`', 'd', '1', '`', 'P', '`', 'l', '11', '`', 'p', '`', 'l', '2', '`', 'P', '`', 'l', '1', '`', 'p', '`', 'l', '2', '`', 'P', '`', 'l', '1', '`', 'p', '`', 'l', '5', '`', 'P', '`', 'l', '1', '`', 'p', '`', 'l', '2', '`', 'P', '`', 'l', '2', '`', 'p', '`', 'l', '6', '`', 'P', '`', 'l', '3', '`', 'p', '`', 'l', '3', '`', 'P', '`', 'l', '2', '`', 'p', '`', 'l', '5', '`', 'P', '`', 'l', '2', '`', 'p', '`', 'l', '9', '`', 'P', '`', 'l', '5', '`', 'p', '`', 'l', '2', '`', 'P', '`', 'l', '2', '`', 'p', '`', 'l', '5', '`', 'P', '`', 'l', '1', '`', 'p', '`', 'l', '2', '`', 'P', '`', 'l', '2', '`', 'p', '`', 'l', '6', '`', 'P', '`', 'l', '17', '`', 'p', '`', 'l', '9', '`', 'P', '`', 'l', '1', '`', 'p', '`', 'l', '5', '`', 'P', '`', 'l', '2', '`', 'p', '`', 'l', '3', '`', 'P', '`', 'l', '5', '`', 'p', '`', 'l', '9', '`', 'P', '`', 'l', '2', '`', 'p', '`', 'l', '2', '`', 'P', '`', 'l', '5', '`', 'p', '`', 'l', '2', '`', 'P', '`', 'l', '2', '`', 'p', '`', 'l', '3', '`', 'P', '`', 'l', '1', '`', 'p', '`', 'l', '6', '`', 'P', '`', 'l', '1', '`', 'p', '`', 'l', '2', '`', 'P', '`', 'l', '2', '`', 'p', '`', 'l', '5', '`', 'P', '`', 'l', '30', '`', 'p', '`', 'l', '2', '`', 'P', '`', 'l', '5', '`', 'p', '`', 'l', '2', '`', 'P', '`', 'l', '2', '`', 'p', '`', 'l', '2', '`', 'P', '`', 'l', '2', '`', 'p', '`', 'l', '5', '`', 'P', '`', 'l', '32', '`', 'p', '`', 'l', '2', '`', 'P', '`', 'l', '2', '`', 'p', '`', 'l', '6', '`', 'P', '`', 'l', '1', '`', 'p', '`', 'l', '3', '`', 'P', '`', 'l', '3', '`', 'p', '`', 'l', '2', '`', 'P', '`', 'l', '3', '`', 'p', '`', 'l', '2', '`', 'P', '`', 'l', '2', '`', 'p', '`', 'l', '6', '`', 'P', '`', 'l', '4', '`', 'p', '`', 'l', '3', '`', 'P', '`', 'l', '1', '`', 'p', '`', 'l', '6', '`', 'P', '`', 'l', '2', '`', 'p', '`', 'l', '8', '`', 'P', '`', 'l', '4', '`', 'p', '`', 'l', '6', '`', 'P', '`', 'l', '11', '`', 'd', '1', '`', 'P', '`', 'r', '10', '`', 'p', '`', 'r', '8', '`', 'P', '`', 'r', '3', '`', 'p', '`', 'r', '9', '`', 'P', '`', 'r', '1', '`', 'p', '`', 'r', '10', '`', 'P', '`', 'r', '4', '`', 'p', '`', 'r', '6', '`', 'P', '`', 'r', '1', '`', 'p', '`', 'r', '3', '`', 'P', '`', 'r', '3', '`', 'p', '`', 'r', '2', '`', 'P', '`', 'r', '3', '`', 'p', '`', 'r', '3', '`', 'P', '`', 'r', '1', '`', 'p', '`', 'r', '10', '`', 'P', '`', 'r', '31', '`', 'p', '`', 'r', '6', '`', 'P', '`', 'r', '2', '`', 'p', '`', 'r', '3', '`', 'P', '`', 'r', '1', '`', 'p', '`', 'r', '2', '`', 'P', '`', 'r', '5', '`', 'p', '`', 'r', '2', '`', 'P', '`', 'r', '29', '`', 'p', '`', 'r', '7', '`', 'P', '`', 'r', '1', '`', 'p', '`', 'r', '2', '`', 'P', '`', 'r', '1', '`', 'p', '`', 'r', '6', '`', 'P', '`', 'r', '1', '`', 'p', '`', 'r', '3', '`', 'P', '`', 'r', '2', '`', 'p', '`', 'r', '2', '`', 'P', '`', 'r', '5', '`', 'p', '`', 'r', '2', '`', 'P', '`', 'r', '2', '`', 'p', '`', 'r', '9', '`', 'P', '`', 'r', '4', '`', 'p', '`', 'r', '5', '`', 'P', '`', 'r', '1', '`', 'p', '`', 'r', '5', '`', 'P', '`', 'r', '1', '`', 'p', '`', 'r', '9', '`', 'P', '`', 'r', '17', '`', 'p', '`', 'r', '6', '`', 'P', '`', 'r', '2', '`', 'p', '`', 'r', '2', '`', 'P', '`', 'r', '2', '`', 'p', '`', 'r', '4', '`', 'P', '`', 'r', '2', '`', 'p', '`', 'r', '2', '`', 'P', '`', 'r', '5', '`', 'p', '`', 'r', '9', '`', 'P', '`', 'r', '1', '`', 'p', '`', 'r', '7', '`', 'P', '`', 'r', '1', '`', 'p', '`', 'r', '3', '`', 'P', '`', 'r', '3', '`', 'p', '`', 'r', '6', '`', 'P', '`', 'r', '2', '`', 'p', '`', 'r', '8', '`', 'P', '`', 'r', '1', '`', 'p', '`', 'r', '2', '`', 'P', '`', 'r', '1', '`', 'p', '`', 'r', '2', '`', 'P', '`', 'r', '11', '`', 'd', '1', '`', 'P', '`', 'l', '11', '`', 'p', '`', 'l', '2', '`', 'P', '`', 'l', '1', '`', 'p', '`', 'l', '2', '`', 'P', '`', 'l', '4', '`', 'p', '`', 'l', '5', '`', 'P', '`', 'l', '2', '`', 'p', '`', 'l', '3', '`', 'P', '`', 'l', '6', '`', 'p', '`', 'l', '3', '`', 'P', '`', 'l', '1', '`', 'p', '`', 'l', '3', '`', 'P', '`', 'l', '5', '`', 'p', '`', 'l', '2', '`', 'P', '`', 'l', '12', '`', 'p', '`', 'l', '2', '`', 'P', '`', 'l', '1', '`', 'p', '`', 'l', '6', '`', 'P', '`', 'l', '1', '`', 'p', '`', 'l', '7', '`', 'P', '`', 'l', '20', '`', 'p', '`', 'l', '2', '`', 'P', '`', 'l', '8', '`', 'p', '`', 'l', '8', '`', 'P', '`', 'l', '1', '`', 'p', '`', 'l', '3', '`', 'P', '`', 'l', '10', '`', 'p', '`', 'l', '2', '`', 'P', '`', 'l', '2', '`', 'p', '`', 'l', '9', '`', 'P', '`', 'l', '2', '`', 'p', '`', 'l', '3', '`', 'P', '`', 'l', '5', '`', 'p', '`', 'l', '2', '`', 'P', '`', 'l', '1', '`', 'p', '`', 'l', '2', '`', 'P', '`', 'l', '1', '`', 'p', '`', 'l', '7', '`', 'P', '`', 'l', '29', '`', 'p', '`', 'l', '13', '`', 'P', '`', 'l', '1', '`', 'p', '`', 'l', '3', '`', 'P', '`', 'l', '2', '`', 'p', '`', 'l', '2', '`', 'P', '`', 'l', '31', '`', 'p', '`', 'l', '6', '`', 'P', '`', 'l', '2', '`', 'p', '`', 'l', '2', '`', 'P', '`', 'l', '1', '`', 'p', '`', 'l', '3', '`', 'P', '`', 'l', '3', '`', 'p', '`', 'l', '2', '`', 'P', '`', 'l', '3', '`', 'p', '`', 'l', '7', '`', 'P', '`', 'l', '6', '`', 'p', '`', 'l', '7', '`', 'P', '`', 'l', '2', '`', 'p', '`', 'l', '2', '`', 'P', '`', 'l', '1', '`', 'p', '`', 'l', '3', '`', 'P', '`', 'l', '8', '`', 'p', '`', 'l', '3', '`', 'P', '`', 'l', '4', '`', 'p', '`', 'l', '3', '`', 'P', '`', 'l', '9', '`', 'd', '1', '`', 'P', '`', 'r', '9', '`', 'p', '`', 'r', '3', '`', 'P', '`', 'r', '4', '`', 'p', '`', 'r', '3', '`', 'P', '`', 'r', '2', '`', 'p', '`', 'r', '9', '`', 'P', '`', 'r', '1', '`', 'p', '`', 'r', '2', '`', 'P', '`', 'r', '2', '`', 'p', '`', 'r', '8', '`', 'P', '`', 'r', '5', '`', 'p', '`', 'r', '3', '`', 'P', '`', 'r', '1', '`', 'p', '`', 'r', '3', '`', 'P', '`', 'r', '3', '`', 'p', '`', 'r', '2', '`', 'P', '`', 'r', '3', '`', 'p', '`', 'r', '3', '`', 'P', '`', 'r', '1', '`', 'p', '`', 'r', '2', '`', 'P', '`', 'r', '2', '`', 'p', '`', 'r', '3', '`', 'P', '`', 'r', '1', '`', 'p', '`', 'r', '2', '`', 'P', '`', 'r', '31', '`', 'p', '`', 'r', '2', '`', 'P', '`', 'r', '3', '`', 'p', '`', 'r', '2', '`', 'P', '`', 'r', '1', '`', 'p', '`', 'r', '2', '`', 'P', '`', 'r', '2', '`', 'p', '`', 'r', '9', '`', 'P', '`', 'r', '30', '`', 'p', '`', 'r', '6', '`', 'P', '`', 'r', '1', '`', 'p', '`', 'r', '2', '`', 'P', '`', 'r', '1', '`', 'p', '`', 'r', '2', '`', 'P', '`', 'r', '5', '`', 'p', '`', 'r', '4', '`', 'P', '`', 'r', '1', '`', 'p', '`', 'r', '9', '`', 'P', '`', 'r', '2', '`', 'p', '`', 'r', '2', '`', 'P', '`', 'r', '10', '`', 'p', '`', 'r', '3', '`', 'P', '`', 'r', '1', '`', 'p', '`', 'r', '8', '`', 'P', '`', 'r', '8', '`', 'p', '`', 'r', '2', '`', 'P', '`', 'r', '19', '`', 'p', '`', 'r', '15', '`', 'P', '`', 'r', '1', '`', 'p', '`', 'r', '2', '`', 'P', '`', 'r', '12', '`', 'p', '`', 'r', '2', '`', 'P', '`', 'r', '5', '`', 'p', '`', 'r', '2', '`', 'P', '`', 'r', '2', '`', 'p', '`', 'r', '3', '`', 'P', '`', 'r', '6', '`', 'p', '`', 'r', '3', '`', 'P', '`', 'r', '2', '`', 'p', '`', 'r', '5', '`', 'P', '`', 'r', '4', '`', 'p', '`', 'r', '2', '`', 'P', '`', 'r', '1', '`', 'p', '`', 'r', '2', '`', 'P', '`', 'r', '11', '`', 'd', '1', '`', 'P', '`', 'l', '11', '`', 'p', '`', 'l', '5', '`', 'P', '`', 'l', '4', '`', 'p', '`', 'l', '5', '`', 'P', '`', 'l', '3', '`', 'p', '`', 'l', '2', '`', 'P', '`', 'l', '6', '`', 'p', '`', 'l', '3', '`', 'P', '`', 'l', '2', '`', 'p', '`', 'l', '3', '`', 'P', '`', 'l', '3', '`', 'p', '`', 'l', '12', '`', 'P', '`', 'l', '1', '`', 'p', '`', 'l', '4', '`', 'P', '`', 'l', '2', '`', 'p', '`', 'l', '2', '`', 'P', '`', 'l', '4', '`', 'p', '`', 'l', '2', '`', 'P', '`', 'l', '2', '`', 'p', '`', 'l', '5', '`', 'P', '`', 'l', '18', '`', 'p', '`', 'l', '2', '`', 'P', '`', 'l', '8', '`', 'p', '`', 'l', '5', '`', 'P', '`', 'l', '1', '`', 'p', '`', 'l', '2', '`', 'P', '`', 'l', '2', '`', 'p', '`', 'l', '2', '`', 'P', '`', 'l', '10', '`', 'p', '`', 'l', '2', '`', 'P', '`', 'l', '2', '`', 'p', '`', 'l', '2', '`', 'P', '`', 'l', '5', '`', 'p', '`', 'l', '7', '`', 'P', '`', 'l', '5', '`', 'p', '`', 'l', '2', '`', 'P', '`', 'l', '1', '`', 'p', '`', 'l', '2', '`', 'P', '`', 'l', '1', '`', 'p', '`', 'l', '2', '`', 'P', '`', 'l', '2', '`', 'p', '`', 'l', '3', '`', 'P', '`', 'l', '29', '`', 'p', '`', 'l', '2', '`', 'P', '`', 'l', '5', '`', 'p', '`', 'l', '6', '`', 'P', '`', 'l', '1', '`', 'p', '`', 'l', '3', '`', 'P', '`', 'l', '1', '`', 'p', '`', 'l', '3', '`', 'P', '`', 'l', '31', '`', 'p', '`', 'l', '6', '`', 'P', '`', 'l', '2', '`', 'p', '`', 'l', '6', '`', 'P', '`', 'l', '3', '`', 'p', '`', 'l', '3', '`', 'P', '`', 'l', '2', '`', 'p', '`', 'l', '8', '`', 'P', '`', 'l', '5', '`', 'p', '`', 'l', '7', '`', 'P', '`', 'l', '2', '`', 'p', '`', 'l', '2', '`', 'P', '`', 'l', '2', '`', 'p', '`', 'l', '9', '`', 'P', '`', 'l', '2', '`', 'p', '`', 'l', '3', '`', 'P', '`', 'l', '3', '`', 'p', '`', 'l', '3', '`', 'P', '`', 'l', '9', '`', 'd', '1', '`', 'P', '`', 'r', '10', '`', 'p', '`', 'r', '8', '`', 'P', '`', 'r', '2', '`', 'p', '`', 'r', '3', '`', 'P', '`', 'r', '8', '`', 'p', '`', 'r', '6', '`', 'P', '`', 'r', '1', '`', 'p', '`', 'r', '3', '`', 'P', '`', 'r', '4', '`', 'p', '`', 'r', '4', '`', 'P', '`', 'r', '1', '`', 'p', '`', 'r', '5', '`', 'P', '`', 'r', '2', '`', 'p', '`', 'r', '4', '`', 'P', '`', 'r', '2', '`', 'p', '`', 'r', '14', '`', 'P', '`', 'r', '32', '`', 'p', '`', 'r', '5', '`', 'P', '`', 'r', '2', '`', 'p', '`', 'r', '13', '`', 'P', '`', 'r', '30', '`', 'p', '`', 'r', '6', '`', 'P', '`', 'r', '1', '`', 'p', '`', 'r', '2', '`', 'P', '`', 'r', '1', '`', 'p', '`', 'r', '10', '`', 'P', '`', 'r', '2', '`', 'p', '`', 'r', '9', '`', 'P', '`', 'r', '2', '`', 'p', '`', 'r', '9', '`', 'P', '`', 'r', '3', '`', 'p', '`', 'r', '2', '`', 'P', '`', 'r', '2', '`', 'p', '`', 'r', '2', '`', 'P', '`', 'r', '1', '`', 'p', '`', 'r', '5', '`', 'P', '`', 'r', '3', '`', 'p', '`', 'r', '3', '`', 'P', '`', 'r', '2', '`', 'p', '`', 'r', '2', '`', 'P', '`', 'r', '16', '`', 'p', '`', 'r', '4', '`', 'P', '`', 'r', '1', '`', 'p', '`', 'r', '3', '`', 'P', '`', 'r', '1', '`', 'p', '`', 'r', '2', '`', 'P', '`', 'r', '3', '`', 'p', '`', 'r', '3', '`', 'P', '`', 'r', '2', '`', 'p', '`', 'r', '4', '`', 'P', '`', 'r', '1', '`', 'p', '`', 'r', '13', '`', 'P', '`', 'r', '1', '`', 'p', '`', 'r', '4', '`', 'P', '`', 'r', '2', '`', 'p', '`', 'r', '3', '`', 'P', '`', 'r', '5', '`', 'p', '`', 'r', '3', '`', 'P', '`', 'r', '3', '`', 'p', '`', 'r', '5', '`', 'P', '`', 'r', '4', '`', 'p', '`', 'r', '5', '`', 'P', '`', 'r', '11', '`', 'd', '1', '`', 'P', '`', 'l', '11', '`', 'p', '`', 'l', '2', '`', 'P', '`', 'l', '1', '`', 'p', '`', 'l', '2', '`', 'P', '`', 'l', '4', '`', 'p', '`', 'l', '5', '`', 'P', '`', 'l', '2', '`', 'p', '`', 'l', '5', '`', 'P', '`', 'l', '4', '`', 'p', '`', 'l', '3', '`', 'P', '`', 'l', '1', '`', 'p', '`', 'l', '6', '`', 'P', '`', 'l', '16', '`', 'p', '`', 'l', '2', '`', 'P', '`', 'l', '2', '`', 'p', '`', 'l', '4', '`', 'P', '`', 'l', '2', '`', 'p', '`', 'l', '2', '`', 'P', '`', 'l', '1', '`', 'p', '`', 'l', '2', '`', 'P', '`', 'l', '3', '`', 'p', '`', 'l', '3', '`', 'P', '`', 'l', '16', '`', 'p', '`', 'l', '3', '`', 'P', '`', 'l', '1', '`', 'p', '`', 'l', '3', '`', 'P', '`', 'l', '3', '`', 'p', '`', 'l', '8', '`', 'P', '`', 'l', '1', '`', 'p', '`', 'l', '3', '`', 'P', '`', 'l', '3', '`', 'p', '`', 'l', '9', '`', 'P', '`', 'l', '2', '`', 'p', '`', 'l', '9', '`', 'P', '`', 'l', '2', '`', 'p', '`', 'l', '10', '`', 'P', '`', 'l', '1', '`', 'p', '`', 'l', '10', '`', 'P', '`', 'l', '29', '`', 'p', '`', 'l', '9', '`', 'P', '`', 'l', '2', '`', 'p', '`', 'l', '2', '`', 'P', '`', 'l', '3', '`', 'p', '`', 'l', '3', '`', 'P', '`', 'l', '33', '`', 'p', '`', 'l', '2', '`', 'P', '`', 'l', '2', '`', 'p', '`', 'l', '6', '`', 'P', '`', 'l', '1', '`', 'p', '`', 'l', '3', '`', 'P', '`', 'l', '1', '`', 'p', '`', 'l', '6', '`', 'P', '`', 'l', '1', '`', 'p', '`', 'l', '2', '`', 'P', '`', 'l', '5', '`', 'p', '`', 'l', '4', '`', 'P', '`', 'l', '3', '`', 'p', '`', 'l', '3', '`', 'P', '`', 'l', '1', '`', 'p', '`', 'l', '5', '`', 'P', '`', 'l', '2', '`', 'p', '`', 'l', '9', '`', 'P', '`', 'l', '5', '`', 'p', '`', 'l', '4', '`', 'P', '`', 'l', '12', '`', 'd', '1', '`', 'P', '`', 'r', '13', '`', 'p', '`', 'r', '2', '`', 'P', '`', 'r', '6', '`', 'p', '`', 'r', '8', '`', 'P', '`', 'r', '3', '`', 'p', '`', 'r', '8', '`', 'P', '`', 'r', '5', '`', 'p', '`', 'r', '1', '`', 'P', '`', 'r', '2', '`', 'p', '`', 'r', '6', '`', 'P', '`', 'r', '2', '`', 'p', '`', 'r', '2', '`', 'P', '`', 'r', '1', '`', 'p', '`', 'r', '3', '`', 'P', '`', 'r', '1', '`', 'p', '`', 'r', '3', '`', 'P', '`', 'r', '9', '`', 'p', '`', 'r', '2', '`', 'P', '`', 'r', '34', '`', 'p', '`', 'r', '6', '`', 'P', '`', 'r', '6', '`', 'p', '`', 'r', '3', '`', 'P', '`', 'r', '31', '`', 'p', '`', 'r', '11', '`', 'P', '`', 'r', '8', '`', 'p', '`', 'r', '3', '`', 'P', '`', 'r', '5', '`', 'p', '`', 'r', '3', '`', 'P', '`', 'r', '8', '`', 'p', '`', 'r', '3', '`', 'P', '`', 'r', '7', '`', 'p', '`', 'r', '11', '`', 'P', '`', 'r', '3', '`', 'p', '`', 'r', '3', '`', 'P', '`', 'r', '1', '`', 'p', '`', 'r', '2', '`', 'P', '`', 'r', '21', '`', 'p', '`', 'r', '6', '`', 'P', '`', 'r', '2', '`', 'p', '`', 'r', '9', '`', 'P', '`', 'r', '5', '`', 'p', '`', 'r', '9', '`', 'P', '`', 'r', '1', '`', 'p', '`', 'r', '3', '`', 'P', '`', 'r', '2', '`', 'p', '`', 'r', '6', '`', 'P', '`', 'r', '3', '`', 'p', '`', 'r', '3', '`', 'P', '`', 'r', '1', '`', 'p', '`', 'r', '3', '`', 'P', '`', 'r', '1', '`', 'p', '`', 'r', '11', '`', 'P', '`', 'r', '1', '`', 'p', '`', 'r', '2', '`', 'P', '`', 'r', '11', '`', 'd', '1', '`', 'P', '`', 'l', '11', '`', 'p', '`', 'l', '2', '`', 'P', '`', 'l', '1', '`', 'p', '`', 'l', '8', '`', 'P', '`', 'l', '1', '`', 'p', '`', 'l', '2', '`', 'P', '`', 'l', '1', '`', 'p', '`', 'l', '2', '`', 'P', '`', 'l', '3', '`', 'p', '`', 'l', '3', '`', 'P', '`', 'l', '2', '`', 'p', '`', 'l', '5', '`', 'P', '`', 'l', '4', '`', 'p', '`', 'l', '2', '`', 'P', '`', 'l', '1', '`', 'p', '`', 'l', '9', '`', 'P', '`', 'l', '5', '`', 'p', '`', 'l', '5', '`', 'P', '`', 'l', '2', '`', 'p', '`', 'l', '3', '`', 'P', '`', 'l', '1', '`', 'p', '`', 'l', '7', '`', 'P', '`', 'l', '18', '`', 'p', '`', 'l', '2', '`', 'P', '`', 'l', '3', '`', 'p', '`', 'l', '4', '`', 'P', '`', 'l', '1', '`', 'p', '`', 'l', '6', '`', 'P', '`', 'l', '2', '`', 'p', '`', 'l', '4', '`', 'P', '`', 'l', '7', '`', 'p', '`', 'l', '3', '`', 'P', '`', 'l', '8', '`', 'p', '`', 'l', '3', '`', 'P', '`', 'l', '5', '`', 'p', '`', 'l', '2', '`', 'P', '`', 'l', '4', '`', 'p', '`', 'l', '3', '`', 'P', '`', 'l', '2', '`', 'p', '`', 'l', '7', '`', 'P', '`', 'l', '1', '`', 'p', '`', 'l', '2', '`', 'P', '`', 'l', '32', '`', 'p', '`', 'l', '3', '`', 'P', '`', 'l', '5', '`', 'p', '`', 'l', '8', '`', 'P', '`', 'l', '33', '`', 'p', '`', 'l', '2', '`', 'P', '`', 'l', '4', '`', 'p', '`', 'l', '2', '`', 'P', '`', 'l', '3', '`', 'p', '`', 'l', '6', '`', 'P', '`', 'l', '2', '`', 'p', '`', 'l', '11', '`', 'P', '`', 'l', '6', '`', 'p', '`', 'l', '9', '`', 'P', '`', 'l', '1', '`', 'p', '`', 'l', '12', '`', 'P', '`', 'l', '1', '`', 'p', '`', 'l', '1', '`', 'P', '`', 'l', '2', '`', 'p', '`', 'l', '2', '`', 'P', '`', 'l', '1', '`', 'p', '`', 'l', '1', '`', 'P', '`', 'l', '11', '`', 'd', '1', '`', 'P', '`', 'r', '8', '`', 'p', '`', 'r', '23', '`', 'P', '`', 'r', '7', '`', 'p', '`', 'r', '3', '`', 'P', '`', 'r', '5', '`', 'p', '`', 'r', '3', '`', 'P', '`', 'r', '3', '`', 'p', '`', 'r', '5', '`', 'P', '`', 'r', '6', '`', 'p', '`', 'r', '3', '`', 'P', '`', 'r', '3', '`', 'p', '`', 'r', '2', '`', 'P', '`', 'r', '38', '`', 'p', '`', 'r', '3', '`', 'P', '`', 'r', '3', '`', 'p', '`', 'r', '16', '`', 'P', '`', 'r', '29', '`', 'p', '`', 'r', '8', '`', 'P', '`', 'r', '2', '`', 'p', '`', 'r', '3', '`', 'P', '`', 'r', '6', '`', 'p', '`', 'r', '24', '`', 'P', '`', 'r', '8', '`', 'p', '`', 'r', '16', '`', 'P', '`', 'r', '17', '`', 'p', '`', 'r', '3', '`', 'P', '`', 'r', '3', '`', 'p', '`', 'r', '5', '`', 'P', '`', 'r', '6', '`', 'p', '`', 'r', '2', '`', 'P', '`', 'r', '5', '`', 'p', '`', 'r', '2', '`', 'P', '`', 'r', '5', '`', 'p', '`', 'r', '2', '`', 'P', '`', 'r', '1', '`', 'p', '`', 'r', '1', '`', 'P', '`', 'r', '7', '`', 'p', '`', 'r', '3', '`', 'P', '`', 'r', '11', '`', 'p', '`', 'r', '2', '`', 'P', '`', 'r', '7', '`', 'p', '`', 'r', '2', '`', 'P', '`', 'r', '1', '`', 'p', '`', 'r', '2', '`', 'P', '`', 'r', '11', '`', 'd', '1', '`', 'P', '`', 'l', '11', '`', 'p', '`', 'l', '2', '`', 'P', '`', 'l', '1', '`', 'p', '`', 'l', '2', '`', 'P', '`', 'l', '7', '`', 'p', '`', 'l', '2', '`', 'P', '`', 'l', '11', '`', 'p', '`', 'l', '3', '`', 'P', '`', 'l', '9', '`', 'p', '`', 'l', '9', '`', 'P', '`', 'l', '5', '`', 'p', '`', 'l', '2', '`', 'P', '`', 'l', '8', '`', 'p', '`', 'l', '3', '`', 'P', '`', 'l', '3', '`', 'p', '`', 'l', '3', '`', 'P', '`', 'l', '17', '`', 'p', '`', 'l', '16', '`', 'P', '`', 'l', '8', '`', 'p', '`', 'l', '24', '`', 'P', '`', 'l', '6', '`', 'p', '`', 'l', '3', '`', 'P', '`', 'l', '2', '`', 'p', '`', 'l', '2', '`', 'P', '`', 'l', '4', '`', 'p', '`', 'l', '3', '`', 'P', '`', 'l', '28', '`', 'p', '`', 'l', '12', '`', 'P', '`', 'l', '1', '`', 'p', '`', 'l', '3', '`', 'P', '`', 'l', '3', '`', 'p', '`', 'l', '2', '`', 'P', '`', 'l', '39', '`', 'p', '`', 'l', '2', '`', 'P', '`', 'l', '3', '`', 'p', '`', 'l', '3', '`', 'P', '`', 'l', '8', '`', 'p', '`', 'l', '3', '`', 'P', '`', 'l', '3', '`', 'p', '`', 'l', '3', '`', 'P', '`', 'l', '5', '`', 'p', '`', 'l', '3', '`', 'P', '`', 'l', '18', '`', 'p', '`', 'l', '12', '`', 'P', '`', 'l', '8', '`', 'd', '1', '`', 'P', '`', 'r', '38', '`', 'p', '`', 'r', '3', '`', 'P', '`', 'r', '6', '`', 'p', '`', 'r', '7', '`', 'P', '`', 'r', '9', '`', 'p', '`', 'r', '3', '`', 'P', '`', 'r', '3', '`', 'p', '`', 'r', '9', '`', 'P', '`', 'r', '32', '`', 'p', '`', 'r', '7', '`', 'P', '`', 'r', '43', '`', 'p', '`', 'r', '8', '`', 'P', '`', 'r', '2', '`', 'p', '`', 'r', '9', '`', 'P', '`', 'r', '32', '`', 'p', '`', 'r', '5', '`', 'P', '`', 'r', '29', '`', 'p', '`', 'r', '7', '`', 'P', '`', 'r', '9', '`', 'p', '`', 'r', '2', '`', 'P', '`', 'r', '5', '`', 'p', '`', 'r', '9', '`', 'P', '`', 'r', '9', '`', 'p', '`', 'r', '3', '`', 'P', '`', 'r', '11', '`', 'p', '`', 'r', '2', '`', 'P', '`', 'r', '7', '`', 'p', '`', 'r', '2', '`', 'P', '`', 'r', '1', '`', 'p', '`', 'r', '2', '`', 'P', '`', 'r', '11', '`', 'd', '1', '`', 'P', '`', 'l', '11', '`', 'p', '`', 'l', '2', '`', 'P', '`', 'l', '10', '`', 'p', '`', 'l', '2', '`', 'P', '`', 'l', '12', '`', 'p', '`', 'l', '2', '`', 'P', '`', 'l', '10', '`', 'p', '`', 'l', '7', '`', 'P', '`', 'l', '6', '`', 'p', '`', 'l', '2', '`', 'P', '`', 'l', '10', '`', 'p', '`', 'l', '5', '`', 'P', '`', 'l', '30', '`', 'p', '`', 'l', '2', '`', 'P', '`', 'l', '35', '`', 'p', '`', 'l', '8', '`', 'P', '`', 'l', '4', '`', 'p', '`', 'l', '6', '`', 'P', '`', 'l', '45', '`', 'p', '`', 'l', '5', '`', 'P', '`', 'l', '34', '`', 'p', '`', 'l', '7', '`', 'P', '`', 'l', '5', '`', 'p', '`', 'l', '2', '`', 'P', '`', 'l', '10', '`', 'p', '`', 'l', '5', '`', 'P', '`', 'l', '7', '`', 'p', '`', 'l', '2', '`', 'P', '`', 'l', '39', '`', 'd', '1', '`', 'P', '`', 'd', '1', '`', 'P', '`', 'd', '1', '`', 'P', '`', 'd', '1', '`', 'P', '`', 'd', '1', '`', 'P', '`', 'd', '1', '`', 'P', '`', 'd', '1', '`', 'P', '`', 'd', '1', '`', 'P', '`', 'd', '1', '`', 'P', '`', 'd', '1', '`', 'P', '`', 'd', '1', '`', 'P', '`', 'd', '1', '`', 'P', '`', 'd', '1', '`', 'P', '`', 'd', '1', '`', 'P', '`', 'd', '1', '`', 'P', '`', 'd', '1', '`', 'P', '`', 'd', '1', '`', 'P', '`', 'd', '1', '`', 'P', '`', 'r', '192', '`', 'p', '`', 'r', '1', '`', 'P', '`', 'r', '54', '`', 'p', '`', 'r', '63', '`', 'P', '`', 'r', '15', '`', 'd', '1', '`', 'P', '`', 'l', '14', '`', 'p', '`', 'l', '65', '`', 'P', '`', 'l', '52', '`', 'p', '`', 'l', '2', '`', 'P', '`', 'l', '192', '`', 'd', '1', '`', 'P', '`', 'r', '191', '`', 'p', '`', 'r', '3', '`', 'P', '`', 'r', '52', '`', 'p', '`', 'r', '65', '`', 'P', '`', 'r', '14', '`', 'd', '1', '`', 'P', '`', 'l', '14', '`', 'p', '`', 'l', '65', '`', 'P', '`', 'l', '51', '`', 'p', '`', 'l', '4', '`', 'P', '`', 'l', '191', '`', 'd', '1', '`', 'P', '`', 'r', '190', '`', 'p', '`', 'r', '5', '`', 'P', '`', 'r', '51', '`', 'p', '`', 'r', '65', '`', 'P', '`', 'r', '14', '`', 'd', '1', '`', 'P', '`', 'l', '14', '`', 'p', '`', 'l', '5', '`', 'P', '`', 'l', '6', '`', 'p', '`', 'l', '4', '`', 'P', '`', 'l', '11', '`', 'p', '`', 'l', '4', '`', 'P', '`', 'l', '10', '`', 'p', '`', 'l', '4', '`', 'P', '`', 'l', '11', '`', 'p', '`', 'l', '4', '`', 'P', '`', 'l', '1', '`', 'p', '`', 'l', '5', '`', 'P', '`', 'l', '50', '`', 'p', '`', 'l', '6', '`', 'P', '`', 'l', '190', '`', 'd', '1', '`', 'P', '`', 'r', '189', '`', 'p', '`', 'r', '7', '`', 'P', '`', 'r', '50', '`', 'p', '`', 'r', '5', '`', 'P', '`', 'r', '2', '`', 'p', '`', 'r', '4', '`', 'P', '`', 'r', '11', '`', 'p', '`', 'r', '4', '`', 'P', '`', 'r', '10', '`', 'p', '`', 'r', '4', '`', 'P', '`', 'r', '11', '`', 'p', '`', 'r', '4', '`', 'P', '`', 'r', '5', '`', 'p', '`', 'r', '5', '`', 'P', '`', 'r', '14', '`', 'd', '1', '`', 'P', '`', 'l', '14', '`', 'p', '`', 'l', '5', '`', 'P', '`', 'l', '4', '`', 'p', '`', 'l', '4', '`', 'P', '`', 'l', '11', '`', 'p', '`', 'l', '4', '`', 'P', '`', 'l', '10', '`', 'p', '`', 'l', '4', '`', 'P', '`', 'l', '11', '`', 'p', '`', 'l', '4', '`', 'P', '`', 'l', '3', '`', 'p', '`', 'l', '5', '`', 'P', '`', 'l', '49', '`', 'p', '`', 'l', '9', '`', 'P', '`', 'l', '188', '`', 'd', '1', '`', 'P', '`', 'r', '188', '`', 'p', '`', 'r', '9', '`', 'P', '`', 'r', '49', '`', 'p', '`', 'r', '5', '`', 'P', '`', 'r', '4', '`', 'p', '`', 'r', '4', '`', 'P', '`', 'r', '11', '`', 'p', '`', 'r', '4', '`', 'P', '`', 'r', '10', '`', 'p', '`', 'r', '4', '`', 'P', '`', 'r', '11', '`', 'p', '`', 'r', '4', '`', 'P', '`', 'r', '3', '`', 'p', '`', 'r', '5', '`', 'P', '`', 'r', '14', '`', 'd', '1', '`', 'P', '`', 'l', '14', '`', 'p', '`', 'l', '5', '`', 'P', '`', 'l', '2', '`', 'p', '`', 'l', '4', '`', 'P', '`', 'l', '11', '`', 'p', '`', 'l', '5', '`', 'P', '`', 'l', '9', '`', 'p', '`', 'l', '4', '`', 'P', '`', 'l', '11', '`', 'p', '`', 'l', '5', '`', 'P', '`', 'l', '4', '`', 'p', '`', 'l', '5', '`', 'P', '`', 'l', '48', '`', 'p', '`', 'l', '11', '`', 'P', '`', 'l', '187', '`', 'd', '1', '`', 'P', '`', 'r', '187', '`', 'p', '`', 'r', '11', '`', 'P', '`', 'r', '48', '`', 'p', '`', 'r', '5', '`', 'P', '`', 'r', '5', '`', 'p', '`', 'r', '5', '`', 'P', '`', 'r', '11', '`', 'p', '`', 'r', '4', '`', 'P', '`', 'r', '9', '`', 'p', '`', 'r', '5', '`', 'P', '`', 'r', '11', '`', 'p', '`', 'r', '4', '`', 'P', '`', 'r', '1', '`', 'p', '`', 'r', '5', '`', 'P', '`', 'r', '14', '`', 'd', '1', '`', 'P', '`', 'l', '14', '`', 'p', '`', 'l', '9', '`', 'P', '`', 'l', '11', '`', 'p', '`', 'l', '5', '`', 'P', '`', 'l', '9', '`', 'p', '`', 'l', '4', '`', 'P', '`', 'l', '11', '`', 'p', '`', 'l', '5', '`', 'P', '`', 'l', '6', '`', 'p', '`', 'l', '5', '`', 'P', '`', 'l', '47', '`', 'p', '`', 'l', '6', '`', 'P', '`', 'l', '1', '`', 'p', '`', 'l', '6', '`', 'P', '`', 'l', '124', '`', 'p', '`', 'l', '1', '`', 'P', '`', 'l', '34', '`', 'p', '`', 'l', '1', '`', 'P', '`', 'l', '26', '`', 'd', '1', '`', 'P', '`', 'r', '25', '`', 'p', '`', 'r', '2', '`', 'P', '`', 'r', '34', '`', 'p', '`', 'r', '1', '`', 'P', '`', 'r', '123', '`', 'p', '`', 'r', '7', '`', 'P', '`', 'r', '1', '`', 'p', '`', 'r', '7', '`', 'P', '`', 'r', '46', '`', 'p', '`', 'r', '5', '`', 'P', '`', 'r', '7', '`', 'p', '`', 'r', '5', '`', 'P', '`', 'r', '11', '`', 'p', '`', 'r', '3', '`', 'P', '`', 'r', '10', '`', 'p', '`', 'r', '4', '`', 'P', '`', 'r', '12', '`', 'p', '`', 'r', '8', '`', 'P', '`', 'r', '14', '`', 'd', '1', '`', 'P', '`', 'l', '14', '`', 'p', '`', 'l', '7', '`', 'P', '`', 'l', '13', '`', 'p', '`', 'l', '3', '`', 'P', '`', 'l', '12', '`', 'p', '`', 'l', '2', '`', 'P', '`', 'l', '10', '`', 'p', '`', 'l', '5', '`', 'P', '`', 'l', '8', '`', 'p', '`', 'l', '5', '`', 'P', '`', 'l', '46', '`', 'p', '`', 'l', '6', '`', 'P', '`', 'l', '3', '`', 'p', '`', 'l', '6', '`', 'P', '`', 'l', '122', '`', 'p', '`', 'l', '2', '`', 'P', '`', 'l', '34', '`', 'p', '`', 'l', '2', '`', 'P', '`', 'l', '25', '`', 'd', '1', '`', 'P', '`', 'r', '25', '`', 'p', '`', 'r', '3', '`', 'P', '`', 'r', '32', '`', 'p', '`', 'r', '3', '`', 'P', '`', 'r', '33', '`', 'p', '`', 'r', '47', '`', 'P', '`', 'r', '41', '`', 'p', '`', 'r', '7', '`', 'P', '`', 'r', '3', '`', 'p', '`', 'r', '7', '`', 'P', '`', 'r', '45', '`', 'p', '`', 'r', '5', '`', 'P', '`', 'r', '9', '`', 'p', '`', 'r', '4', '`', 'P', '`', 'r', '25', '`', 'p', '`', 'r', '1', '`', 'P', '`', 'r', '15', '`', 'p', '`', 'r', '6', '`', 'P', '`', 'r', '14', '`', 'd', '1', '`', 'P', '`', 'l', '14', '`', 'p', '`', 'l', '6', '`', 'P', '`', 'l', '42', '`', 'p', '`', 'l', '2', '`', 'P', '`', 'l', '10', '`', 'p', '`', 'l', '5', '`', 'P', '`', 'l', '45', '`', 'p', '`', 'l', '6', '`', 'P', '`', 'l', '5', '`', 'p', '`', 'l', '6', '`', 'P', '`', 'l', '41', '`', 'p', '`', 'l', '48', '`', 'P', '`', 'l', '32', '`', 'p', '`', 'l', '3', '`', 'P', '`', 'l', '32', '`', 'p', '`', 'l', '3', '`', 'P', '`', 'l', '25', '`', 'd', '1', '`', 'P', '`', 'r', '24', '`', 'p', '`', 'r', '4', '`', 'P', '`', 'r', '32', '`', 'p', '`', 'r', '4', '`', 'P', '`', 'r', '32', '`', 'p', '`', 'r', '47', '`', 'P', '`', 'r', '40', '`', 'p', '`', 'r', '7', '`', 'P', '`', 'r', '5', '`', 'p', '`', 'r', '7', '`', 'P', '`', 'r', '44', '`', 'p', '`', 'r', '6', '`', 'P', '`', 'r', '54', '`', 'p', '`', 'r', '5', '`', 'P', '`', 'r', '14', '`', 'd', '1', '`', 'P', '`', 'l', '14', '`', 'p', '`', 'l', '5', '`', 'P', '`', 'l', '53', '`', 'p', '`', 'l', '7', '`', 'P', '`', 'l', '44', '`', 'p', '`', 'l', '6', '`', 'P', '`', 'l', '7', '`', 'p', '`', 'l', '6', '`', 'P', '`', 'l', '41', '`', 'p', '`', 'l', '46', '`', 'P', '`', 'l', '32', '`', 'p', '`', 'l', '5', '`', 'P', '`', 'l', '31', '`', 'p', '`', 'l', '4', '`', 'P', '`', 'l', '24', '`', 'd', '1', '`', 'P', '`', 'r', '24', '`', 'p', '`', 'r', '5', '`', 'P', '`', 'r', '30', '`', 'p', '`', 'r', '5', '`', 'P', '`', 'r', '33', '`', 'p', '`', 'r', '45', '`', 'P', '`', 'r', '40', '`', 'p', '`', 'r', '6', '`', 'P', '`', 'r', '8', '`', 'p', '`', 'r', '7', '`', 'P', '`', 'r', '43', '`', 'p', '`', 'r', '7', '`', 'P', '`', 'r', '45', '`', 'p', '`', 'r', '2', '`', 'P', '`', 'r', '6', '`', 'p', '`', 'r', '5', '`', 'P', '`', 'r', '14', '`', 'd', '1', '`', 'P', '`', 'l', '14', '`', 'p', '`', 'l', '5', '`', 'P', '`', 'l', '5', '`', 'p', '`', 'l', '4', '`', 'P', '`', 'l', '43', '`', 'p', '`', 'l', '8', '`', 'P', '`', 'l', '43', '`', 'p', '`', 'l', '6', '`', 'P', '`', 'l', '9', '`', 'p', '`', 'l', '7', '`', 'P', '`', 'l', '40', '`', 'p', '`', 'l', '6', '`', 'P', '`', 'l', '31', '`', 'p', '`', 'l', '7', '`', 'P', '`', 'l', '33', '`', 'p', '`', 'l', '5', '`', 'P', '`', 'l', '30', '`', 'p', '`', 'l', '6', '`', 'P', '`', 'l', '23', '`', 'd', '1', '`', 'P', '`', 'r', '23', '`', 'p', '`', 'r', '6', '`', 'P', '`', 'r', '29', '`', 'p', '`', 'r', '7', '`', 'P', '`', 'r', '33', '`', 'p', '`', 'r', '6', '`', 'P', '`', 'r', '30', '`', 'p', '`', 'r', '7', '`', 'P', '`', 'r', '40', '`', 'p', '`', 'r', '6', '`', 'P', '`', 'r', '10', '`', 'p', '`', 'r', '7', '`', 'P', '`', 'r', '42', '`', 'p', '`', 'r', '9', '`', 'P', '`', 'r', '43', '`', 'p', '`', 'r', '4', '`', 'P', '`', 'r', '4', '`', 'p', '`', 'r', '5', '`', 'P', '`', 'r', '14', '`', 'd', '1', '`', 'P', '`', 'l', '14', '`', 'p', '`', 'l', '5', '`', 'P', '`', 'l', '3', '`', 'p', '`', 'l', '4', '`', 'P', '`', 'l', '43', '`', 'p', '`', 'l', '4', '`', 'P', '`', 'l', '1', '`', 'p', '`', 'l', '5', '`', 'P', '`', 'l', '42', '`', 'p', '`', 'l', '6', '`', 'P', '`', 'l', '11', '`', 'p', '`', 'l', '7', '`', 'P', '`', 'l', '40', '`', 'p', '`', 'l', '6', '`', 'P', '`', 'l', '29', '`', 'p', '`', 'l', '7', '`', 'P', '`', 'l', '33', '`', 'p', '`', 'l', '7', '`', 'P', '`', 'l', '28', '`', 'p', '`', 'l', '7', '`', 'P', '`', 'l', '23', '`', 'd', '1', '`', 'P', '`', 'r', '22', '`', 'p', '`', 'r', '8', '`', 'P', '`', 'r', '28', '`', 'p', '`', 'r', '7', '`', 'P', '`', 'r', '34', '`', 'p', '`', 'r', '6', '`', 'P', '`', 'r', '28', '`', 'p', '`', 'r', '7', '`', 'P', '`', 'r', '40', '`', 'p', '`', 'r', '6', '`', 'P', '`', 'r', '13', '`', 'p', '`', 'r', '6', '`', 'P', '`', 'r', '41', '`', 'p', '`', 'r', '5', '`', 'P', '`', 'r', '2', '`', 'p', '`', 'r', '4', '`', 'P', '`', 'r', '43', '`', 'p', '`', 'r', '4', '`', 'P', '`', 'r', '2', '`', 'p', '`', 'r', '5', '`', 'P', '`', 'r', '14', '`', 'd', '1', '`', 'P', '`', 'l', '14', '`', 'p', '`', 'l', '5', '`', 'P', '`', 'l', '1', '`', 'p', '`', 'l', '4', '`', 'P', '`', 'l', '43', '`', 'p', '`', 'l', '4', '`', 'P', '`', 'l', '3', '`', 'p', '`', 'l', '5', '`', 'P', '`', 'l', '41', '`', 'p', '`', 'l', '6', '`', 'P', '`', 'l', '13', '`', 'p', '`', 'l', '7', '`', 'P', '`', 'l', '40', '`', 'p', '`', 'l', '6', '`', 'P', '`', 'l', '28', '`', 'p', '`', 'l', '6', '`', 'P', '`', 'l', '33', '`', 'p', '`', 'l', '8', '`', 'P', '`', 'l', '28', '`', 'p', '`', 'l', '8', '`', 'P', '`', 'l', '22', '`', 'd', '1', '`', 'P', '`', 'r', '22', '`', 'p', '`', 'r', '9', '`', 'P', '`', 'r', '26', '`', 'p', '`', 'r', '9', '`', 'P', '`', 'r', '34', '`', 'p', '`', 'r', '6', '`', 'P', '`', 'r', '26', '`', 'p', '`', 'r', '7', '`', 'P', '`', 'r', '40', '`', 'p', '`', 'r', '6', '`', 'P', '`', 'r', '15', '`', 'p', '`', 'r', '6', '`', 'P', '`', 'r', '40', '`', 'p', '`', 'r', '5', '`', 'P', '`', 'r', '4', '`', 'p', '`', 'r', '4', '`', 'P', '`', 'r', '42', '`', 'p', '`', 'r', '4', '`', 'P', '`', 'r', '1', '`', 'p', '`', 'r', '5', '`', 'P', '`', 'r', '14', '`', 'd', '1', '`', 'P', '`', 'l', '14', '`', 'p', '`', 'l', '9', '`', 'P', '`', 'l', '43', '`', 'p', '`', 'l', '3', '`', 'P', '`', 'l', '5', '`', 'p', '`', 'l', '5', '`', 'P', '`', 'l', '40', '`', 'p', '`', 'l', '6', '`', 'P', '`', 'l', '16', '`', 'p', '`', 'l', '6', '`', 'P', '`', 'l', '40', '`', 'p', '`', 'l', '6', '`', 'P', '`', 'l', '26', '`', 'p', '`', 'l', '6', '`', 'P', '`', 'l', '34', '`', 'p', '`', 'l', '9', '`', 'P', '`', 'l', '26', '`', 'p', '`', 'l', '9', '`', 'P', '`', 'l', '22', '`', 'd', '1', '`', 'P', '`', 'r', '21', '`', 'p', '`', 'r', '10', '`', 'P', '`', 'r', '26', '`', 'p', '`', 'r', '9', '`', 'P', '`', 'r', '35', '`', 'p', '`', 'r', '6', '`', 'P', '`', 'r', '24', '`', 'p', '`', 'r', '7', '`', 'P', '`', 'r', '39', '`', 'p', '`', 'r', '7', '`', 'P', '`', 'r', '17', '`', 'p', '`', 'r', '6', '`', 'P', '`', 'r', '39', '`', 'p', '`', 'r', '5', '`', 'P', '`', 'r', '6', '`', 'p', '`', 'r', '2', '`', 'P', '`', 'r', '44', '`', 'p', '`', 'r', '8', '`', 'P', '`', 'r', '14', '`', 'd', '1', '`', 'P', '`', 'l', '14', '`', 'p', '`', 'l', '7', '`', 'P', '`', 'l', '53', '`', 'p', '`', 'l', '5', '`', 'P', '`', 'l', '39', '`', 'p', '`', 'l', '6', '`', 'P', '`', 'l', '18', '`', 'p', '`', 'l', '6', '`', 'P', '`', 'l', '40', '`', 'p', '`', 'l', '6', '`', 'P', '`', 'l', '24', '`', 'p', '`', 'l', '6', '`', 'P', '`', 'l', '34', '`', 'p', '`', 'l', '11', '`', 'P', '`', 'l', '25', '`', 'p', '`', 'l', '10', '`', 'P', '`', 'l', '21', '`', 'd', '1', '`', 'P', '`', 'r', '21', '`', 'p', '`', 'r', '11', '`', 'P', '`', 'r', '24', '`', 'p', '`', 'r', '11', '`', 'P', '`', 'r', '35', '`', 'p', '`', 'r', '6', '`', 'P', '`', 'r', '23', '`', 'p', '`', 'r', '6', '`', 'P', '`', 'r', '39', '`', 'p', '`', 'r', '7', '`', 'P', '`', 'r', '19', '`', 'p', '`', 'r', '6', '`', 'P', '`', 'r', '38', '`', 'p', '`', 'r', '5', '`', 'P', '`', 'r', '54', '`', 'p', '`', 'r', '6', '`', 'P', '`', 'r', '14', '`', 'd', '1', '`', 'P', '`', 'l', '14', '`', 'p', '`', 'l', '5', '`', 'P', '`', 'l', '54', '`', 'p', '`', 'l', '6', '`', 'P', '`', 'l', '38', '`', 'p', '`', 'l', '6', '`', 'P', '`', 'l', '20', '`', 'p', '`', 'l', '6', '`', 'P', '`', 'l', '40', '`', 'p', '`', 'l', '6', '`', 'P', '`', 'l', '22', '`', 'p', '`', 'l', '6', '`', 'P', '`', 'l', '35', '`', 'p', '`', 'l', '11', '`', 'P', '`', 'l', '24', '`', 'p', '`', 'l', '12', '`', 'P', '`', 'l', '20', '`', 'd', '1', '`', 'P', '`', 'r', '20', '`', 'p', '`', 'r', '12', '`', 'P', '`', 'r', '23', '`', 'p', '`', 'r', '13', '`', 'P', '`', 'r', '35', '`', 'p', '`', 'r', '6', '`', 'P', '`', 'r', '21', '`', 'p', '`', 'r', '6', '`', 'P', '`', 'r', '39', '`', 'p', '`', 'r', '6', '`', 'P', '`', 'r', '22', '`', 'p', '`', 'r', '6', '`', 'P', '`', 'r', '37', '`', 'p', '`', 'r', '7', '`', 'P', '`', 'r', '45', '`', 'p', '`', 'r', '2', '`', 'P', '`', 'r', '6', '`', 'p', '`', 'r', '5', '`', 'P', '`', 'r', '14', '`', 'd', '1', '`', 'P', '`', 'l', '14', '`', 'p', '`', 'l', '5', '`', 'P', '`', 'l', '6', '`', 'p', '`', 'l', '3', '`', 'P', '`', 'l', '43', '`', 'p', '`', 'l', '8', '`', 'P', '`', 'l', '37', '`', 'p', '`', 'l', '6', '`', 'P', '`', 'l', '22', '`', 'p', '`', 'l', '6', '`', 'P', '`', 'l', '40', '`', 'p', '`', 'l', '6', '`', 'P', '`', 'l', '20', '`', 'p', '`', 'l', '6', '`', 'P', '`', 'l', '35', '`', 'p', '`', 'l', '13', '`', 'P', '`', 'l', '22', '`', 'p', '`', 'l', '13', '`', 'P', '`', 'l', '20', '`', 'd', '1', '`', 'P', '`', 'r', '19', '`', 'p', '`', 'r', '14', '`', 'P', '`', 'r', '22', '`', 'p', '`', 'r', '13', '`', 'P', '`', 'r', '36', '`', 'p', '`', 'r', '6', '`', 'P', '`', 'r', '19', '`', 'p', '`', 'r', '6', '`', 'P', '`', 'r', '39', '`', 'p', '`', 'r', '6', '`', 'P', '`', 'r', '24', '`', 'p', '`', 'r', '6', '`', 'P', '`', 'r', '36', '`', 'p', '`', 'r', '9', '`', 'P', '`', 'r', '42', '`', 'p', '`', 'r', '4', '`', 'P', '`', 'r', '5', '`', 'p', '`', 'r', '5', '`', 'P', '`', 'r', '14', '`', 'd', '1', '`', 'P', '`', 'l', '14', '`', 'p', '`', 'l', '5', '`', 'P', '`', 'l', '4', '`', 'p', '`', 'l', '4', '`', 'P', '`', 'l', '42', '`', 'p', '`', 'l', '4', '`', 'P', '`', 'l', '1', '`', 'p', '`', 'l', '5', '`', 'P', '`', 'l', '35', '`', 'p', '`', 'l', '7', '`', 'P', '`', 'l', '24', '`', 'p', '`', 'l', '7', '`', 'P', '`', 'l', '39', '`', 'p', '`', 'l', '6', '`', 'P', '`', 'l', '18', '`', 'p', '`', 'l', '6', '`', 'P', '`', 'l', '35', '`', 'p', '`', 'l', '14', '`', 'P', '`', 'l', '22', '`', 'p', '`', 'l', '14', '`', 'P', '`', 'l', '19', '`', 'd', '1', '`', 'P', '`', 'r', '19', '`', 'p', '`', 'r', '14', '`', 'P', '`', 'r', '21', '`', 'p', '`', 'r', '15', '`', 'P', '`', 'r', '36', '`', 'p', '`', 'r', '6', '`', 'P', '`', 'r', '17', '`', 'p', '`', 'r', '6', '`', 'P', '`', 'r', '39', '`', 'p', '`', 'r', '6', '`', 'P', '`', 'r', '26', '`', 'p', '`', 'r', '6', '`', 'P', '`', 'r', '35', '`', 'p', '`', 'r', '5', '`', 'P', '`', 'r', '1', '`', 'p', '`', 'r', '4', '`', 'P', '`', 'r', '43', '`', 'p', '`', 'r', '4', '`', 'P', '`', 'r', '3', '`', 'p', '`', 'r', '5', '`', 'P', '`', 'r', '14', '`', 'd', '1', '`', 'P', '`', 'l', '14', '`', 'p', '`', 'l', '5', '`', 'P', '`', 'l', '2', '`', 'p', '`', 'l', '4', '`', 'P', '`', 'l', '43', '`', 'p', '`', 'l', '4', '`', 'P', '`', 'l', '2', '`', 'p', '`', 'l', '5', '`', 'P', '`', 'l', '35', '`', 'p', '`', 'l', '6', '`', 'P', '`', 'l', '25', '`', 'p', '`', 'l', '6', '`', 'P', '`', 'l', '41', '`', 'p', '`', 'l', '6', '`', 'P', '`', 'l', '16', '`', 'p', '`', 'l', '6', '`', 'P', '`', 'l', '36', '`', 'p', '`', 'l', '15', '`', 'P', '`', 'l', '20', '`', 'p', '`', 'l', '15', '`', 'P', '`', 'l', '19', '`', 'd', '1', '`', 'P', '`', 'r', '18', '`', 'p', '`', 'r', '16', '`', 'P', '`', 'r', '20', '`', 'p', '`', 'r', '15', '`', 'P', '`', 'r', '37', '`', 'p', '`', 'r', '5', '`', 'P', '`', 'r', '16', '`', 'p', '`', 'r', '6', '`', 'P', '`', 'r', '41', '`', 'p', '`', 'r', '6', '`', 'P', '`', 'r', '25', '`', 'p', '`', 'r', '6', '`', 'P', '`', 'r', '35', '`', 'p', '`', 'r', '5', '`', 'P', '`', 'r', '3', '`', 'p', '`', 'r', '4', '`', 'P', '`', 'r', '43', '`', 'p', '`', 'r', '4', '`', 'P', '`', 'r', '1', '`', 'p', '`', 'r', '5', '`', 'P', '`', 'r', '14', '`', 'd', '1', '`', 'P', '`', 'l', '14', '`', 'p', '`', 'l', '9', '`', 'P', '`', 'l', '43', '`', 'p', '`', 'l', '4', '`', 'P', '`', 'l', '4', '`', 'p', '`', 'l', '5', '`', 'P', '`', 'l', '35', '`', 'p', '`', 'l', '7', '`', 'P', '`', 'l', '23', '`', 'p', '`', 'l', '6', '`', 'P', '`', 'l', '43', '`', 'p', '`', 'l', '6', '`', 'P', '`', 'l', '14', '`', 'p', '`', 'l', '6', '`', 'P', '`', 'l', '36', '`', 'p', '`', 'l', '17', '`', 'P', '`', 'l', '19', '`', 'p', '`', 'l', '16', '`', 'P', '`', 'l', '18', '`', 'd', '1', '`', 'P', '`', 'r', '18', '`', 'p', '`', 'r', '17', '`', 'P', '`', 'r', '18', '`', 'p', '`', 'r', '17', '`', 'P', '`', 'r', '37', '`', 'p', '`', 'r', '5', '`', 'P', '`', 'r', '14', '`', 'p', '`', 'r', '6', '`', 'P', '`', 'r', '43', '`', 'p', '`', 'r', '7', '`', 'P', '`', 'r', '22', '`', 'p', '`', 'r', '6', '`', 'P', '`', 'r', '36', '`', 'p', '`', 'r', '5', '`', 'P', '`', 'r', '5', '`', 'p', '`', 'r', '3', '`', 'P', '`', 'r', '44', '`', 'p', '`', 'r', '8', '`', 'P', '`', 'r', '14', '`', 'd', '1', '`', 'P', '`', 'l', '14', '`', 'p', '`', 'l', '7', '`', 'P', '`', 'l', '46', '`', 'p', '`', 'l', '1', '`', 'P', '`', 'l', '6', '`', 'p', '`', 'l', '5', '`', 'P', '`', 'l', '37', '`', 'p', '`', 'l', '6', '`', 'P', '`', 'l', '21', '`', 'p', '`', 'l', '6', '`', 'P', '`', 'l', '45', '`', 'p', '`', 'l', '6', '`', 'P', '`', 'l', '12', '`', 'p', '`', 'l', '6', '`', 'P', '`', 'l', '37', '`', 'p', '`', 'l', '17', '`', 'P', '`', 'l', '18', '`', 'p', '`', 'l', '18', '`', 'P', '`', 'l', '17', '`', 'd', '1', '`', 'P', '`', 'r', '17', '`', 'p', '`', 'r', '18', '`', 'P', '`', 'r', '17', '`', 'p', '`', 'r', '19', '`', 'P', '`', 'r', '37', '`', 'p', '`', 'r', '5', '`', 'P', '`', 'r', '12', '`', 'p', '`', 'r', '6', '`', 'P', '`', 'r', '45', '`', 'p', '`', 'r', '7', '`', 'P', '`', 'r', '20', '`', 'p', '`', 'r', '6', '`', 'P', '`', 'r', '37', '`', 'p', '`', 'r', '5', '`', 'P', '`', 'r', '53', '`', 'p', '`', 'r', '7', '`', 'P', '`', 'r', '14', '`', 'd', '1', '`', 'P', '`', 'l', '14', '`', 'p', '`', 'l', '6', '`', 'P', '`', 'l', '53', '`', 'p', '`', 'l', '6', '`', 'P', '`', 'l', '38', '`', 'p', '`', 'l', '6', '`', 'P', '`', 'l', '19', '`', 'p', '`', 'l', '6', '`', 'P', '`', 'l', '47', '`', 'p', '`', 'l', '5', '`', 'P', '`', 'l', '11', '`', 'p', '`', 'l', '6', '`', 'P', '`', 'l', '37', '`', 'p', '`', 'l', '19', '`', 'P', '`', 'l', '16', '`', 'p', '`', 'l', '19', '`', 'P', '`', 'l', '17', '`', 'd', '1', '`', 'P', '`', 'r', '16', '`', 'p', '`', 'r', '20', '`', 'P', '`', 'r', '16', '`', 'p', '`', 'r', '19', '`', 'P', '`', 'r', '38', '`', 'p', '`', 'r', '5', '`', 'P', '`', 'r', '10', '`', 'p', '`', 'r', '6', '`', 'P', '`', 'r', '47', '`', 'p', '`', 'r', '7', '`', 'P', '`', 'r', '18', '`', 'p', '`', 'r', '6', '`', 'P', '`', 'r', '38', '`', 'p', '`', 'r', '7', '`', 'P', '`', 'r', '45', '`', 'p', '`', 'r', '2', '`', 'P', '`', 'r', '6', '`', 'p', '`', 'r', '5', '`', 'P', '`', 'r', '14', '`', 'd', '1', '`', 'P', '`', 'l', '14', '`', 'p', '`', 'l', '5', '`', 'P', '`', 'l', '5', '`', 'p', '`', 'l', '4', '`', 'P', '`', 'l', '43', '`', 'p', '`', 'l', '8', '`', 'P', '`', 'l', '39', '`', 'p', '`', 'l', '6', '`', 'P', '`', 'l', '17', '`', 'p', '`', 'l', '6', '`', 'P', '`', 'l', '49', '`', 'p', '`', 'l', '5', '`', 'P', '`', 'l', '9', '`', 'p', '`', 'l', '6', '`', 'P', '`', 'l', '37', '`', 'p', '`', 'l', '20', '`', 'P', '`', 'l', '16', '`', 'p', '`', 'l', '20', '`', 'P', '`', 'l', '16', '`', 'd', '1', '`', 'P', '`', 'r', '16', '`', 'p', '`', 'r', '20', '`', 'P', '`', 'r', '15', '`', 'p', '`', 'r', '21', '`', 'P', '`', 'r', '38', '`', 'p', '`', 'r', '5', '`', 'P', '`', 'r', '8', '`', 'p', '`', 'r', '6', '`', 'P', '`', 'r', '49', '`', 'p', '`', 'r', '7', '`', 'P', '`', 'r', '15', '`', 'p', '`', 'r', '7', '`', 'P', '`', 'r', '39', '`', 'p', '`', 'r', '9', '`', 'P', '`', 'r', '42', '`', 'p', '`', 'r', '4', '`', 'P', '`', 'r', '5', '`', 'p', '`', 'r', '5', '`', 'P', '`', 'r', '14', '`', 'd', '1', '`', 'P', '`', 'l', '14', '`', 'p', '`', 'l', '5', '`', 'P', '`', 'l', '4', '`', 'p', '`', 'l', '4', '`', 'P', '`', 'l', '42', '`', 'p', '`', 'l', '4', '`', 'P', '`', 'l', '1', '`', 'p', '`', 'l', '5', '`', 'P', '`', 'l', '40', '`', 'p', '`', 'l', '6', '`', 'P', '`', 'l', '15', '`', 'p', '`', 'l', '6', '`', 'P', '`', 'l', '51', '`', 'p', '`', 'l', '5', '`', 'P', '`', 'l', '7', '`', 'p', '`', 'l', '6', '`', 'P', '`', 'l', '38', '`', 'p', '`', 'l', '21', '`', 'P', '`', 'l', '14', '`', 'p', '`', 'l', '21', '`', 'P', '`', 'l', '16', '`', 'd', '1', '`', 'P', '`', 'r', '15', '`', 'p', '`', 'r', '22', '`', 'P', '`', 'r', '14', '`', 'p', '`', 'r', '21', '`', 'P', '`', 'r', '39', '`', 'p', '`', 'r', '5', '`', 'P', '`', 'r', '6', '`', 'p', '`', 'r', '6', '`', 'P', '`', 'r', '52', '`', 'p', '`', 'r', '6', '`', 'P', '`', 'r', '13', '`', 'p', '`', 'r', '7', '`', 'P', '`', 'r', '40', '`', 'p', '`', 'r', '5', '`', 'P', '`', 'r', '2', '`', 'p', '`', 'r', '4', '`', 'P', '`', 'r', '42', '`', 'p', '`', 'r', '4', '`', 'P', '`', 'r', '3', '`', 'p', '`', 'r', '5', '`', 'P', '`', 'r', '14', '`', 'd', '1', '`', 'P', '`', 'l', '14', '`', 'p', '`', 'l', '5', '`', 'P', '`', 'l', '2', '`', 'p', '`', 'l', '4', '`', 'P', '`', 'l', '43', '`', 'p', '`', 'l', '4', '`', 'P', '`', 'l', '2', '`', 'p', '`', 'l', '5', '`', 'P', '`', 'l', '41', '`', 'p', '`', 'l', '6', '`', 'P', '`', 'l', '13', '`', 'p', '`', 'l', '6', '`', 'P', '`', 'l', '53', '`', 'p', '`', 'l', '5', '`', 'P', '`', 'l', '5', '`', 'p', '`', 'l', '6', '`', 'P', '`', 'l', '38', '`', 'p', '`', 'l', '23', '`', 'P', '`', 'l', '13', '`', 'p', '`', 'l', '22', '`', 'P', '`', 'l', '15', '`', 'd', '1', '`', 'P', '`', 'r', '15', '`', 'p', '`', 'r', '23', '`', 'P', '`', 'r', '12', '`', 'p', '`', 'r', '23', '`', 'P', '`', 'r', '39', '`', 'p', '`', 'r', '5', '`', 'P', '`', 'r', '4', '`', 'p', '`', 'r', '6', '`', 'P', '`', 'r', '54', '`', 'p', '`', 'r', '6', '`', 'P', '`', 'r', '11', '`', 'p', '`', 'r', '7', '`', 'P', '`', 'r', '41', '`', 'p', '`', 'r', '5', '`', 'P', '`', 'r', '3', '`', 'p', '`', 'r', '4', '`', 'P', '`', 'r', '43', '`', 'p', '`', 'r', '4', '`', 'P', '`', 'r', '1', '`', 'p', '`', 'r', '5', '`', 'P', '`', 'r', '14', '`', 'd', '1', '`', 'P', '`', 'l', '14', '`', 'p', '`', 'l', '9', '`', 'P', '`', 'l', '43', '`', 'p', '`', 'l', '4', '`', 'P', '`', 'l', '4', '`', 'p', '`', 'l', '5', '`', 'P', '`', 'l', '42', '`', 'p', '`', 'l', '6', '`', 'P', '`', 'l', '11', '`', 'p', '`', 'l', '6', '`', 'P', '`', 'l', '55', '`', 'p', '`', 'l', '5', '`', 'P', '`', 'l', '3', '`', 'p', '`', 'l', '6', '`', 'P', '`', 'l', '39', '`', 'p', '`', 'l', '23', '`', 'P', '`', 'l', '12', '`', 'p', '`', 'l', '24', '`', 'P', '`', 'l', '14', '`', 'd', '1', '`', 'P', '`', 'r', '14', '`', 'p', '`', 'r', '24', '`', 'P', '`', 'r', '11', '`', 'p', '`', 'r', '25', '`', 'P', '`', 'r', '39', '`', 'p', '`', 'r', '5', '`', 'P', '`', 'r', '2', '`', 'p', '`', 'r', '6', '`', 'P', '`', 'r', '56', '`', 'p', '`', 'r', '6', '`', 'P', '`', 'r', '9', '`', 'p', '`', 'r', '7', '`', 'P', '`', 'r', '42', '`', 'p', '`', 'r', '5', '`', 'P', '`', 'r', '5', '`', 'p', '`', 'r', '3', '`', 'P', '`', 'r', '44', '`', 'p', '`', 'r', '8', '`', 'P', '`', 'r', '14', '`', 'd', '1', '`', 'P', '`', 'l', '14', '`', 'p', '`', 'l', '7', '`', 'P', '`', 'l', '46', '`', 'p', '`', 'l', '1', '`', 'P', '`', 'l', '6', '`', 'p', '`', 'l', '5', '`', 'P', '`', 'l', '43', '`', 'p', '`', 'l', '6', '`', 'P', '`', 'l', '9', '`', 'p', '`', 'l', '6', '`', 'P', '`', 'l', '57', '`', 'p', '`', 'l', '5', '`', 'P', '`', 'l', '2', '`', 'p', '`', 'l', '5', '`', 'P', '`', 'l', '39', '`', 'p', '`', 'l', '25', '`', 'P', '`', 'l', '10', '`', 'p', '`', 'l', '25', '`', 'P', '`', 'l', '14', '`', 'd', '1', '`', 'P', '`', 'r', '13', '`', 'p', '`', 'r', '26', '`', 'P', '`', 'r', '10', '`', 'p', '`', 'r', '25', '`', 'P', '`', 'r', '40', '`', 'p', '`', 'r', '11', '`', 'P', '`', 'r', '58', '`', 'p', '`', 'r', '6', '`', 'P', '`', 'r', '7', '`', 'p', '`', 'r', '6', '`', 'P', '`', 'r', '44', '`', 'p', '`', 'r', '5', '`', 'P', '`', 'r', '54', '`', 'p', '`', 'r', '6', '`', 'P', '`', 'r', '14', '`', 'd', '1', '`', 'P', '`', 'l', '14', '`', 'p', '`', 'l', '6', '`', 'P', '`', 'l', '54', '`', 'p', '`', 'l', '5', '`', 'P', '`', 'l', '44', '`', 'p', '`', 'l', '6', '`', 'P', '`', 'l', '7', '`', 'p', '`', 'l', '6', '`', 'P', '`', 'l', '59', '`', 'p', '`', 'l', '10', '`', 'P', '`', 'l', '39', '`', 'p', '`', 'l', '26', '`', 'P', '`', 'l', '10', '`', 'p', '`', 'l', '26', '`', 'P', '`', 'l', '13', '`', 'd', '1', '`', 'P', '`', 'r', '13', '`', 'p', '`', 'r', '26', '`', 'P', '`', 'r', '9', '`', 'p', '`', 'r', '27', '`', 'P', '`', 'r', '40', '`', 'p', '`', 'r', '9', '`', 'P', '`', 'r', '60', '`', 'p', '`', 'r', '6', '`', 'P', '`', 'r', '5', '`', 'p', '`', 'r', '6', '`', 'P', '`', 'r', '45', '`', 'p', '`', 'r', '5', '`', 'P', '`', 'r', '55', '`', 'p', '`', 'r', '5', '`', 'P', '`', 'r', '14', '`', 'd', '1', '`', 'P', '`', 'l', '14', '`', 'p', '`', 'l', '5', '`', 'P', '`', 'l', '54', '`', 'p', '`', 'l', '6', '`', 'P', '`', 'l', '45', '`', 'p', '`', 'l', '6', '`', 'P', '`', 'l', '4', '`', 'p', '`', 'l', '7', '`', 'P', '`', 'l', '61', '`', 'p', '`', 'l', '8', '`', 'P', '`', 'l', '40', '`', 'p', '`', 'l', '27', '`', 'P', '`', 'l', '8', '`', 'p', '`', 'l', '27', '`', 'P', '`', 'l', '13', '`', 'd', '1', '`', 'P', '`', 'r', '12', '`', 'p', '`', 'r', '28', '`', 'P', '`', 'r', '8', '`', 'p', '`', 'r', '27', '`', 'P', '`', 'r', '41', '`', 'p', '`', 'r', '7', '`', 'P', '`', 'r', '62', '`', 'p', '`', 'r', '6', '`', 'P', '`', 'r', '3', '`', 'p', '`', 'r', '6', '`', 'P', '`', 'r', '46', '`', 'p', '`', 'r', '7', '`', 'P', '`', 'r', '10', '`', 'p', '`', 'r', '2', '`', 'P', '`', 'r', '14', '`', 'p', '`', 'r', '1', '`', 'P', '`', 'r', '26', '`', 'p', '`', 'r', '5', '`', 'P', '`', 'r', '14', '`', 'd', '1', '`', 'P', '`', 'l', '14', '`', 'p', '`', 'l', '5', '`', 'P', '`', 'l', '8', '`', 'p', '`', 'l', '3', '`', 'P', '`', 'l', '14', '`', 'p', '`', 'l', '4', '`', 'P', '`', 'l', '11', '`', 'p', '`', 'l', '4', '`', 'P', '`', 'l', '8', '`', 'p', '`', 'l', '8', '`', 'P', '`', 'l', '46', '`', 'p', '`', 'l', '7', '`', 'P', '`', 'l', '1', '`', 'p', '`', 'l', '7', '`', 'P', '`', 'l', '63', '`', 'p', '`', 'l', '6', '`', 'P', '`', 'l', '40', '`', 'p', '`', 'l', '29', '`', 'P', '`', 'l', '7', '`', 'p', '`', 'l', '28', '`', 'P', '`', 'l', '12', '`', 'd', '1', '`', 'P', '`', 'r', '12', '`', 'p', '`', 'r', '29', '`', 'P', '`', 'r', '6', '`', 'p', '`', 'r', '29', '`', 'P', '`', 'r', '41', '`', 'p', '`', 'r', '5', '`', 'P', '`', 'r', '64', '`', 'p', '`', 'r', '6', '`', 'P', '`', 'r', '1', '`', 'p', '`', 'r', '6', '`', 'P', '`', 'r', '47', '`', 'p', '`', 'r', '9', '`', 'P', '`', 'r', '8', '`', 'p', '`', 'r', '4', '`', 'P', '`', 'r', '11', '`', 'p', '`', 'r', '4', '`', 'P', '`', 'r', '13', '`', 'p', '`', 'r', '4', '`', 'P', '`', 'r', '7', '`', 'p', '`', 'r', '5', '`', 'P', '`', 'r', '14', '`', 'd', '1', '`', 'P', '`', 'l', '14', '`', 'p', '`', 'l', '5', '`', 'P', '`', 'l', '5', '`', 'p', '`', 'l', '5', '`', 'P', '`', 'l', '12', '`', 'p', '`', 'l', '5', '`', 'P', '`', 'l', '10', '`', 'p', '`', 'l', '5', '`', 'P', '`', 'l', '8', '`', 'p', '`', 'l', '10', '`', 'P', '`', 'l', '47', '`', 'p', '`', 'l', '13', '`', 'P', '`', 'l', '65', '`', 'p', '`', 'l', '4', '`', 'P', '`', 'l', '41', '`', 'p', '`', 'l', '29', '`', 'P', '`', 'l', '6', '`', 'p', '`', 'l', '30', '`', 'P', '`', 'l', '11', '`', 'd', '1', '`', 'P', '`', 'r', '11', '`', 'p', '`', 'r', '30', '`', 'P', '`', 'r', '5', '`', 'p', '`', 'r', '31', '`', 'P', '`', 'r', '41', '`', 'p', '`', 'r', '3', '`', 'P', '`', 'r', '66', '`', 'p', '`', 'r', '11', '`', 'P', '`', 'r', '48', '`', 'p', '`', 'r', '5', '`', 'P', '`', 'r', '1', '`', 'p', '`', 'r', '5', '`', 'P', '`', 'r', '8', '`', 'p', '`', 'r', '5', '`', 'P', '`', 'r', '10', '`', 'p', '`', 'r', '5', '`', 'P', '`', 'r', '12', '`', 'p', '`', 'r', '5', '`', 'P', '`', 'r', '4', '`', 'p', '`', 'r', '5', '`', 'P', '`', 'r', '14', '`', 'd', '1', '`', 'P', '`', 'l', '14', '`', 'p', '`', 'l', '5', '`', 'P', '`', 'l', '3', '`', 'p', '`', 'l', '5', '`', 'P', '`', 'l', '12', '`', 'p', '`', 'l', '5', '`', 'P', '`', 'l', '10', '`', 'p', '`', 'l', '5', '`', 'P', '`', 'l', '8', '`', 'p', '`', 'l', '4', '`', 'P', '`', 'l', '3', '`', 'p', '`', 'l', '5', '`', 'P', '`', 'l', '48', '`', 'p', '`', 'l', '11', '`', 'P', '`', 'l', '67', '`', 'p', '`', 'l', '2', '`', 'P', '`', 'l', '42', '`', 'p', '`', 'l', '29', '`', 'P', '`', 'l', '6', '`', 'p', '`', 'l', '30', '`', 'P', '`', 'l', '11', '`', 'd', '1', '`', 'P', '`', 'r', '188', '`', 'p', '`', 'r', '9', '`', 'P', '`', 'r', '49', '`', 'p', '`', 'r', '5', '`', 'P', '`', 'r', '4', '`', 'p', '`', 'r', '4', '`', 'P', '`', 'r', '9', '`', 'p', '`', 'r', '4', '`', 'P', '`', 'r', '11', '`', 'p', '`', 'r', '4', '`', 'P', '`', 'r', '13', '`', 'p', '`', 'r', '4', '`', 'P', '`', 'r', '2', '`', 'p', '`', 'r', '5', '`', 'P', '`', 'r', '14', '`', 'd', '1', '`', 'P', '`', 'l', '14', '`', 'p', '`', 'l', '10', '`', 'P', '`', 'l', '12', '`', 'p', '`', 'l', '5', '`', 'P', '`', 'l', '10', '`', 'p', '`', 'l', '5', '`', 'P', '`', 'l', '9', '`', 'p', '`', 'l', '4', '`', 'P', '`', 'l', '5', '`', 'p', '`', 'l', '5', '`', 'P', '`', 'l', '49', '`', 'p', '`', 'l', '8', '`', 'P', '`', 'l', '189', '`', 'd', '1', '`', 'P', '`', 'r', '189', '`', 'p', '`', 'r', '7', '`', 'P', '`', 'r', '50', '`', 'p', '`', 'r', '65', '`', 'P', '`', 'r', '14', '`', 'd', '1', '`', 'P', '`', 'l', '14', '`', 'p', '`', 'l', '65', '`', 'P', '`', 'l', '51', '`', 'p', '`', 'l', '5', '`', 'P', '`', 'l', '190', '`', 'd', '1', '`', 'P', '`', 'r', '190', '`', 'p', '`', 'r', '5', '`', 'P', '`', 'r', '51', '`', 'p', '`', 'r', '65', '`', 'P', '`', 'r', '14', '`', 'd', '1', '`', 'P', '`', 'l', '14', '`', 'p', '`', 'l', '65', '`', 'P', '`', 'l', '52', '`', 'p', '`', 'l', '3', '`', 'P', '`', 'l', '191', '`', 'd', '1', '`', 'P', '`', 'r', '191', '`', 'p', '`', 'r', '3', '`', 'P', '`', 'r', '54', '`', 'p', '`', 'r', '2', '`', 'P', '`', 'r', '10', '`', 'p', '`', 'r', '4', '`', 'P', '`', 'r', '6', '`', 'p', '`', 'r', '1', '`', 'P', '`', 'r', '3', '`', 'p', '`', 'r', '2', '`', 'P', '`', 'r', '9', '`', 'p', '`', 'r', '6', '`', 'P', '`', 'r', '10', '`', 'p', '`', 'r', '2', '`', 'P', '`', 'r', '22', '`', 'd', '1', '`', 'P', '`', 'l', '132', '`', 'p', '`', 'l', '1', '`', 'P', '`', 'l', '192', '`', 'd', '1', '`', 'P', '`', 'd', '1', '`', 'P', '`', 'd', '1', '`', 'P', '`', 'd', '1', '`', 'P', '`', 'd', '1', '`', 'P', '`', 'd', '1', '`', 'P', '`', 'd', '1', '`', 'P', '`', 'd', '1', '`', 'P', '`', 'd', '1', '`', 'P', '`', 'd', '1', '`', 'P', '`', 'd', '1', '`', 'P', '`', 'd', '1', '`', 'P', '`', 'd', '1', '`', 'P', '`', 'd', '1', '`', 'P', '`', 'd', '1', '`', 'P', '`', 'd', '1', '`', 'P', '`', 'd', '1', '`', 'P', '`', 'd', '1', '`', 'P', '`', 'd', '1', '`', 'P', '`', 'd', '1', '`', 'P', '`', 'd', '1', '`', 'P', '`', 'd', '1', '`', 'P', '`', 'd', '1', '`', 'P', '`', 'd', '1', '`', 'P', '`', 'd', '1', '`', 'P', '`', 'd', '1', '`', 'P', '`', 'd', '1', '`', 'P', '`', 'd', '1', '`', 'P', '`', 'd', '1', '`', 'P', '`', 'd', '1', '`', 'P', '`', 'd', '1', '`', 'P', '`', 'd', '1', '`', 'P', '`', 'd', '1', '`', 'P', '`', 'd', '1', '`', 'P', '`', 'd', '1', '`', 'P', '`', 'd', '1', '`', 'P', '`', 'd', '1', '`', 'P', '`', 'd', '1', '`', 'P', '`', 'd', '1', '`', 'P', '`', 'd', '1', '`', 'P', '`', 'd', '1', '`', 'P', '`', 'd', '1', '`', 'P', '`', 'd', '1', '`', 'P', '`', 'd', '1', '`', 'P', '`', 'd', '1', '`', 'P', '`', 'd', '1', '`', 'P', '`', 'd', '1', '`', 'P', '`', 'd', '1', '`', 'P', '`', 'd', '1', '`', 'P', '`', 'd', '1', '`', 'P', '`', 'd', '1', '`', 'P', '`', 'r', '290', '`', 'p', '`', 'r', '6', '`', 'P', '`', 'r', '5', '`', 'p', '`', 'r', '4', '`', 'P', '`', 'r', '20', '`', 'd', '1', '`', 'P', '`', 'l', '20', '`', 'p', '`', 'l', '4', '`', 'P', '`', 'l', '5', '`', 'p', '`', 'l', '2', '`', 'P', '`', 'l', '1', '`', 'p', '`', 'l', '2', '`', 'P', '`', 'l', '291', '`', 'd', '1', '`', 'P', '`', 'r', '291', '`', 'p', '`', 'r', '7', '`', 'P', '`', 'r', '1', '`', 'p', '`', 'r', '14', '`', 'P', '`', 'r', '1', '`', 'p', '`', 'r', '4', '`', 'P', '`', 'r', '1', '`', 'p', '`', 'r', '4', '`', 'P', '`', 'r', '2', '`', 'd', '1', '`', 'P', '`', 'l', '2', '`', 'p', '`', 'l', '7', '`', 'P', '`', 'l', '1', '`', 'p', '`', 'l', '1', '`', 'P', '`', 'l', '1', '`', 'p', '`', 'l', '1', '`', 'P', '`', 'l', '1', '`', 'p', '`', 'l', '2', '`', 'P', '`', 'l', '2', '`', 'p', '`', 'l', '2', '`', 'P', '`', 'l', '1', '`', 'p', '`', 'l', '1', '`', 'P', '`', 'l', '1', '`', 'p', '`', 'l', '1', '`', 'P', '`', 'l', '1', '`', 'p', '`', 'l', '1', '`', 'P', '`', 'l', '1', '`', 'p', '`', 'l', '7', '`', 'P', '`', 'l', '291', '`', 'd', '1', '`', 'P', '`', 'r', '291', '`', 'p', '`', 'r', '1', '`', 'P', '`', 'r', '2', '`', 'p', '`', 'r', '4', '`', 'P', '`', 'r', '1', '`', 'p', '`', 'r', '1', '`', 'P', '`', 'r', '1', '`', 'p', '`', 'r', '1', '`', 'P', '`', 'r', '1', '`', 'p', '`', 'r', '1', '`', 'P', '`', 'r', '2', '`', 'p', '`', 'r', '2', '`', 'P', '`', 'r', '1', '`', 'p', '`', 'r', '1', '`', 'P', '`', 'r', '2', '`', 'p', '`', 'r', '3', '`', 'P', '`', 'r', '1', '`', 'p', '`', 'r', '4', '`', 'P', '`', 'r', '1', '`', 'p', '`', 'r', '1', '`', 'P', '`', 'r', '3', '`', 'd', '1', '`', 'P', '`', 'l', '3', '`', 'p', '`', 'l', '4', '`', 'P', '`', 'l', '1', '`', 'p', '`', 'l', '1', '`', 'P', '`', 'l', '1', '`', 'p', '`', 'l', '6', '`', 'P', '`', 'l', '1', '`', 'p', '`', 'l', '3', '`', 'P', '`', 'l', '1', '`', 'p', '`', 'l', '1', '`', 'P', '`', 'l', '1', '`', 'p', '`', 'l', '1', '`', 'P', '`', 'l', '1', '`', 'p', '`', 'l', '4', '`', 'P', '`', 'l', '1', '`', 'p', '`', 'l', '5', '`', 'P', '`', 'l', '290', '`', 'd', '1', '`', 'P', '`', 'd', '1', '`', 'P', '`', 'd', '1', '`', 'P', '`', 'r', '297', '`', 'p', '`', 'r', '1', '`', 'P', '`', 'r', '4', '`', 'p', '`', 'r', '1', '`', 'P', '`', 'r', '2', '`', 'p', '`', 'r', '3', '`', 'P', '`', 'r', '3', '`', 'p', '`', 'r', '3', '`', 'P', '`', 'r', '11', '`', 'd', '1', '`', 'P', '`', 'l', '8', '`', 'p', '`', 'l', '3', '`', 'P', '`', 'l', '2', '`', 'p', '`', 'l', '1', '`', 'P', '`', 'l', '1', '`', 'p', '`', 'l', '6', '`', 'P', '`', 'l', '1', '`', 'p', '`', 'l', '7', '`', 'P', '`', 'l', '1', '`', 'p', '`', 'l', '1', '`', 'P', '`', 'l', '294', '`', 'd', '1', '`', 'P', '`', 'r', '294', '`', 'p', '`', 'r', '4', '`', 'P', '`', 'r', '1', '`', 'p', '`', 'r', '1', '`', 'P', '`', 'r', '1', '`', 'p', '`', 'r', '2', '`', 'P', '`', 'r', '1', '`', 'p', '`', 'r', '3', '`', 'P', '`', 'r', '1', '`', 'p', '`', 'r', '1', '`', 'P', '`', 'r', '2', '`', 'p', '`', 'r', '6', '`', 'P', '`', 'r', '8', '`', 'd', '1', '`', 'P', '`', 'l', '16', '`', 'p', '`', 'l', '1', '`', 'P', '`', 'l', '308', '`', 'd', '1', '`', 'P', '`', 'd', '1', '`'])

def interact_ser(_str, _ard):
    _ard.write(_str.encode())
    tmp = _ard.readline()
    print(tmp.decode())
    
if __name__ == "__main__":
    port = 'COM4'  # 변동가능
    ard = serial.Serial(port, 9600)
    time.sleep(3)

    for i in serial_deque:
        if list(pyautogui.position()) != [0, 0]:
            interact_ser(i, ard)

    ard.close()


In [ ]:
print(a)